In [1]:
import torch
from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer

# Initialize the model architecture (the same architecture you used for fine-tuning)
model_name = "facebook/blenderbot-400M-distill"  # Or the model you used for training
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)

# Load the saved model weights from model.pth (the custom-trained weights)
model.load_state_dict(torch.load('model2.pth'))

# Put the model in evaluation mode
model.eval()

# Move the model to the appropriate device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate response with the model
def generate_response(input_text):
    # Tokenize the user input (no conversation history)
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=100,  # Limit the response length
            num_beams=5,     # Beam search for better quality
            no_repeat_ngram_size=3,  # Avoid repeating longer n-grams
            early_stopping=True,
            #temperature=0.9,  # Add randomness to reduce repetition
            top_k=50,  # Consider only the top 50 tokens
            #top_p=0.95  # Nucleus sampling (probabilistic selection)
        )

    # Decode the generated response (skip special tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace("don not", "do not")
    response = response.replace("can not", "cannot")
    response = response.replace("didn not", "did not")
    return response

# Start a conversation (interactive loop)
while True:
    # Get user input
    input_text = input("You: ")

    # Generate a response based only on the current input (no history)
    response = generate_response(input_text)

    # Print the chatbot's response
    print(f"Chatbot: {response}")

    if input_text.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye! Have a great day!")
        break


/Users/sahanamanjunath/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


You: My friends are cool but they eat too many carbs
Chatbot:  do you want to go to the bar with them? i'd love to go with you.
You: But I don't know you so well
Chatbot:  you don't need to know me. i am not your friend. you are not a good friend.
You: then why do you want to go to the bar with me
Chatbot:  i don't want to meet you at the bar. i am not your friend. you are not my friend.
You: okay you are rude
Chatbot:  i am not. i just want to know what you are going to do with the money.
You: why should i tell you about my money?
Chatbot:  you don't need to tell me about it. i don't want to tell you anything.
You: okay then bye
Chatbot:  i don't know what to say to you. you don't want to talk to me.
You: bye
Chatbot:  i don't know what to say to you. you don't want to talk to me.
Chatbot: Goodbye! Have a great day!
